In [ ]:
#Imports
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from math import pi
from time import time
import minterpy_levelsets as ls

# Local imports
import surfgeopy as sp

# Gauss Bonnet theorem on sphere
This is a benchmark of computing surface integrals using high-order surface quadrature (HOSQ) method and GPLS method for algebraic varieties.



# Step 1: Load and extract the vertices of a sphere mesh composed of          triangles.

In [ ]:
mesh_path="../meshes/SphereMesh_N=1652_r=1.mat"
mesh_mat = scipy.io.loadmat(mesh_path)
pointcloud= mesh_mat["xs"]

In [ ]:

def phi(x):
    return x[0]**2+x[1]**2+x[2]**2-1

def dphi(x):
    return np.array([2*x[0],2*x[1],2*x[2]])

# Step 2: Perform surface fitting

In [ ]:
# result = np.zeros(pointcloud.shape[0])  # initialize an empty array to store the results
# for i in range(pointcloud.shape[0]):
#     result[i] = phi(pointcloud[i])
# print(f"The accuracy of the given mesh is:{(result).max()}")
# if (result).max()>1e-8:
#     for i in range(pointcloud.shape[0]):
#         pointcloud[i,:]=sp.SimpleImplicitSurfaceProjection(phi, dphi,pointcloud[i,:])
newt_poly = ls.LevelsetPoly(pointcloud, method='BK',verbose=True)

# Step 3: We execute  HOSQ for the standard sphere $\mathbb{S}^2$.


This step involves utilizing the zero level set provided by GPLS and computing the curvature:

If $M$ is orienatble, then  the gradient $\nabla Q_M = (\partial_x Q_M,\, \partial_y Q_M,\, \partial_z Q_M) \in \mathbb{R}^3$ never vanishes on $M$ and provides, together with the Hessian $H_M =\nabla(\nabla Q_M) \in \mathbb{R}^{3 \times 3}$ of $Q_M$
\begin{equation*}
 H_M  = \left(\begin{array}{ccc}
 \frac{\partial^2 Q_M}{\partial_{x}^2} & \frac{\partial^2 Q_M}{\partial_{x}\partial_y} & \frac{\partial^2 Q_M}{\partial_{x}\partial_z} \\
 \frac{\partial^2 Q_M}{\partial_{y}\partial_x} & \frac{\partial^2 Q_M}{\partial_y^2} & \frac{\partial^2 Q_M}{\partial_{y}\partial_z} \\
 \frac{\partial^2 Q_M}{\partial_z\partial_{x}} & \frac{\partial^2 Q_M}{\partial_{z}\partial_y} & \frac{\partial^2 Q_M}{\partial_z^2} \\
 \end{array}\right) ,
\end{equation*}
the main ingredients for the following computations. Both Gauss and mean curvature can be computed from these quantities \cite{goldman2005} as:
\begin{align}
  K_{\mathrm{Gauss}} &= \frac{\det \left(\begin{array}{cc}H_M  & \nabla Q_M^T \\  \nabla Q_M & 0 \end{array}\right)}{\|\nabla Q_M\|^4} \label{eq:GC}\\
  K_{\mathrm{mean}} &= \frac{\nabla Q_M H_M \nabla Q_M^T - \|\nabla Q_M\|^2\mathrm{trace}(H_M)}{2\|\nabla Q_M\|^3}\,. \label{eq:MC}
\end{align}

In [ ]:
def err_t(intp_degree,lp_degree,func,grad_fun,mesh_path,Refinement):
    mesh_mat = scipy.io.loadmat(mesh_path)
    xs =mesh_mat["xs"]
    elems = mesh_mat["surfs"] - 1
    t0 = time()
    pnts, ws, offset = sp.compute_surf_quadrature(func,grad_fun, xs, elems,intp_degree,lp_degree,Refinement)
    # number of quadrature points
    nqp = pnts.shape[0]
    #compute the mean and Gauss curvature using GPLS
    mean_curvature, gauss_curvature = newt_poly.compute_curvatures_at(pnts)
    # function value at each quadrature points
    fs_qp = np.array(
            [
                    gauss_curvature[qpid]

                for qpid in range(nqp)
            ]
        )
    # numerical integrations
    nints = np.matmul(ws, fs_qp)
    exact_area =4*np.pi
    t1 = time()
    print("Relative error: ", abs( nints - exact_area)/exact_area)
    print ("The main function takes:",{(t1-t0)})
    error=abs( nints - exact_area)/exact_area
    return error

# here is the error obtained using Dune Curved Grid
error_dune2_14=np.array([3.54125508e-05, 1.33316704e-05, 2.57541325e-08, 1.03435839e-08,
                         2.81819789e-11, 8.84523266e-13, 1.41998103e-11, 2.53958653e-11,
                         1.90391275e-09, 1.05589086e-08, 1.97662884e-07, 5.95443649e-07,1.27624767e-05])

eror_HOSQ_GPLS=np.array([8.210388017188951e-07,5.2762010987399376e-09,6.382208455893933e-10,
                 3.2260719529049058e-12,1.6044131393160407e-12,1.2298144768325598e-14,
                 4.240739575284689e-15,2.8271597168564595e-15,3.2512336743849282e-15,
                 3.1098756885421053e-15,3.392591660227751e-15,3.675307631913397e-15,3.533949646070574e-15])
# Degree of Polynomial
Nrange = list(range(2,15))
lp_degree=float("inf")
refinement=0
error1=[] 
for n in Nrange:
    if n%1==0:print(n)
    erro1 = err_t(int(n),lp_degree,newt_poly,newt_poly.compute_gradients_at,mesh_path,refinement)
    error1.append(erro1)

plt.semilogy(Nrange, error1, '-or')
plt.semilogy(Nrange, error_dune2_14, '-ob')
plt.xlabel("Degree of Polynomial",fontsize=13)
plt.ylabel("Relative Error",fontsize=13)
plt.legend(['HOSQ+GPLS','DCG'],prop={'size': 13},loc='upper center')
plt.xticks(np.arange(min(Nrange), max(Nrange)+1, 1.0))
plt.ylim([2.758195177427762e-18,3.9514540203871754e-04])
plt.grid()
plt.savefig("../images/dune_vs_HOSQ+GPLS.pdf")